# Definizioni iniziali

#### Pacchetti

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

#### Funzioni di misura

In [ ]:
def mutual_information(img1: np.ndarray, img2: np.ndarray, bins: int = 256) -> float:
    """
    Compute the mutual information (MI) between two grayscale images.
    
    :param img1: First image in grayscale.
    :param img2: Second image in grayscale.
    :param bins: Number of histogram bins to use (256 for 8-bit images).
    :return: Mutual information (in bits) between img1 and img2.
    """
    # Convert images to 1D numpy arrays (flatten) if they are not already
    img1 = img1.ravel()
    img2 = img2.ravel()

    # Compute the joint histogram
    # histogram2d returns the 2D histogram and bin edges
    joint_hist, x_edges, y_edges = np.histogram2d(img1, img2, bins=bins)
    
    # Normalize the joint histogram to get the joint PDF p(x,y)
    joint_pdf = joint_hist / np.sum(joint_hist)
    
    # Compute the marginal PDFs p(x) and p(y)
    p_x = np.sum(joint_pdf, axis=1)  # Sum over columns -> marginal over x
    p_y = np.sum(joint_pdf, axis=0)  # Sum over rows -> marginal over y

    # Only consider non-zero values to avoid log(0)
    # MI = sum p(x,y)*log( p(x,y)/(p(x)*p(y)) )
    non_zero_idxs = joint_pdf > 0
    mi = np.sum(joint_pdf[non_zero_idxs] * 
                np.log2(joint_pdf[non_zero_idxs] / 
                        (p_x[np.newaxis].T @ p_y[np.newaxis])[non_zero_idxs]))
    
    return mi

#### Funzione di allineamento

Controllo dei diversi metodi per applicare CLAHE alle immagini originali

In [ ]:
clipLimits = [10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0]
tileGridSizes = [(4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12)]

highest = (None, None, 0)
for clipLimit in clipLimits:
    for tileGridSize in tileGridSizes:
        label_free_clahe = apply_clahe(label_free_gray, clipLimit=clipLimit, tileGridSize=tileGridSize)
        stained_clahe = apply_clahe(stained_gray, clipLimit=clipLimit, tileGridSize=tileGridSize)
        _, score = align(label_free_clahe, stained_clahe, image_to_align=stained_gray, image_to_confront=label_free_gray, mode=MODE_ORB, use_lowes_ratio=False, use_euclidean_distance=True)
        print(f"CLAHE\tCL: {clipLimit}\tTGS: {tileGridSize}\tSCORE: {score}")
        if score > highest[2]:
            highest = (clipLimit, tileGridSize, score)
print(f"BEST - CL: {highest[0]} - TGS: {highest[1]} - SCORE: {highest[2]}")